In [0]:
import torch
import numpy as np
from torchvision.datasets import mnist

from torch import nn
from torch.autograd import Variable

In [0]:
train_Set = mnist.MNIST('./data', train=True, download=True)
test_Set = mnist.MNIST('./data', train=False, download=True)

In [0]:
a_data, a_label = train_Set[0]

In [4]:
a_data

In [5]:
a_label

5

In [6]:
a_data = np.array(a_data, dtype='float32')
print(a_data.shape)

(28, 28)


In [7]:
print(a_data)

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   3.  18.
   18.  18. 126. 136. 175.  26. 166. 255. 247. 127.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.  30.  36.  94. 154. 170. 253.
  253. 253. 253. 253. 225. 172. 253. 242. 195.  64.   0.  

In [0]:
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5
    x = x.reshape((-1,))
    x = torch.from_numpy(x)
    return x

train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True)

test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)

In [9]:
a, a_label = train_set[0]
print(a.shape)
print(a_label)

torch.Size([784])
5


In [0]:
from torch.utils.data import DataLoader

train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

a, a_label = next(iter(train_data))

In [11]:
print(a.shape)
print(a_label.shape)

torch.Size([64, 784])
torch.Size([64])


In [0]:
net = nn.Sequential(
    nn.Linear(784, 400),
    nn.ReLU(),
    nn.Linear(400, 200),
    nn.ReLU(),
    nn.Linear(200, 100),
    nn.ReLU(),
    nn.Linear(100,10)
)

In [13]:
net

Sequential(
  (0): Linear(in_features=784, out_features=400, bias=True)
  (1): ReLU()
  (2): Linear(in_features=400, out_features=200, bias=True)
  (3): ReLU()
  (4): Linear(in_features=200, out_features=100, bias=True)
  (5): ReLU()
  (6): Linear(in_features=100, out_features=10, bias=True)
)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 1e-1)

In [15]:
losses = []
acces = []
eval_losses = []
eval_acces = []

for e in range(20):
    train_loss = 0
    train_acc = 0
    #？？？？？？
    net.train()
    for im, label in train_data:
        im = Variable(im)
        label = Variable(label)
        
        out = net(im)
        loss = criterion(out, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data[0]
        
        _, pred = out.max(1)
        num_correct = (pred == label).sum().data[0]
        acc = num_correct / im.shape[0]
        
        train_acc += acc
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    
    eval_loss = 0
    eval_acc = 0
    net.eval()
    
    for im, label in test_data:
        im = Variable(im)
        label = Variable(label)
        out = net(im)
        loss = criterion(out, label)
        
        eval_loss += loss.data[0]
        _, pred = out.max(1)
        
        num_correct = (pred == label).sum().data[0]
        acc = num_correct / im.shape[0]
        eval_acc += acc
    
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    
    print('epoch:{}, Train Loss:{:.6f}, Train Acc: {:.6f}, Eval Loss:{:.6f}, Eval Acc:{:.6f}'.format(e, train_loss/ len(train_data), train_acc / len(train_data), eval_loss / len(test_data), eval_acc / len(test_data)))
        

IndexError: ignored

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
plt.title('train loss')
plt.plot(np.arange(len(losses)), losses)

In [0]:
plt.title('train acc')
plt.plot(np.arange(len(acces)),acces)

In [0]:
plt.plot(np.arange(len(eval_losses)), eval_losses)
plt.title('test loss')

In [0]:
plt.plot(np.arange(len(eval_acces)), eval_acces)
plt.title('test acc')